In [5]:
#Importo as bibliotecas
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import os
import time


class ScoresFixtures:
    
    def __init__(self, link):
        self.link = link
        req = requests.get(link)
        if req.status_code == 200:
            content = req.content
        #pegando a base do site
        soup = BeautifulSoup(content, 'html.parser')
        tb = soup.find(id="sched_ks_3260_1")

        s1= []
        s2= []
        for i in tb.find_all("a"):
                s1.append(str(i))
                s2.append(str(i.get_text('href')))


        # Calling DataFrame constructor after zipping 
        # both lists, with columns specified 
        di = pd.DataFrame(list(zip(s1, s2)), 
                       columns =['Codes', 'ID']) 

        s4=[]
        for i in di["Codes"]:
            i = i.replace('<a href="','')
            i = i.replace('</a>','')
            s4.append(str(i))


        s5 = []

        for i in di['Codes']:
            if "matches" in i:
                s5.append(str(i))
            else:
                s5.append(0)

        s6 = []
        for i in di["Codes"]:
            if '<a href="/en/squads/' in i:
                i = i.replace('<a href="/en/squads/','')
                i = i[0:8]
                s6.append(str(i))
            else:
                s6.append(0)        

        # Calling DataFrame constructor after zipping 
        # both lists, with columns specified 
        da = pd.DataFrame(list(zip(s1, s2,s4,s5,s6)), 
                       columns =['CODES', 'ID','URL_FINAL','PARTIDAS_2019',"TEAM_CODE"])        

        s9 = []
        for i in da["URL_FINAL"]:
            if 'Match Report' in i:
                s9.append(str(i))
            else:
                pass
        self.table =  s9



In [6]:
def planilhas(url):
    #faço a request
    pg = 'https://fbref.com'
    url_pg = pg+ url
    req = requests.get(url_pg)
    if req.status_code == 200:
        content = req.content
    #pegando a base do site
    soup = BeautifulSoup(content, 'html.parser')

    table_geral = soup.find_all(class_ = "table_outer_container")
    table_time_1 = table_geral[0]
    table_time_2 = table_geral[7]
    table_time_3 = soup.find(class_='venuetime')

    # nova parte


    table_torcida = soup.find_all('div',class_ = "scorebox_meta")
    oi_torcida = str(table_torcida)



    toby = oi_torcida.split('<small>')
    torcida = str(toby[2])
    torcida = torcida.split('</small>')
    torcida = str(torcida[0])
    estadio = toby[4]
    estadio = estadio.split('</small>')
    estadio = str(estadio[0])    


    #conseguindo a data do jogo
    data = table_time_3.get('data-venue-date')        

    #tratando o nome da planilha
    nome = str(soup.title)
    nome = nome.replace(" ","_")
    nome = nome.replace("<title>","")
    nome = nome.replace(".","")
    nome_final = nome.split("Report")[0]


    #retratando o nome final

    nome_final = nome_final.split("_Match")
    nome_final = nome_final[0]    


    # transofmrando a base do site em string e fazendo algumas tratativas
    table_str_1 = str(table_time_1)
    table_str_2 = str(table_time_2)
    df_1 = pd.read_html(table_str_1)[0]
    df_2 = pd.read_html(table_str_2)[0]



    time = str(nome_final)
    time = time.replace("_"," ")
    time = time.split(" vs ")
    time_1 = str(time[0])
    time_2 = str(time[1])

    #trnasformando em dataframe
    df_1 = pd.DataFrame(df_1)
    df_1.columns = df_1.columns.droplevel()
    df_1['Time'] = str(time_1)
    df_1['Time_Adversario'] = str(time_2)
    df_1['Confronto'] = str(nome_final)
    df_1['Data'] = str(data)
    df_1['Estadio'] = str(estadio)
    df_1['Torcida'] = torcida    


    df_2 = pd.DataFrame(df_2)
    df_2.columns = df_2.columns.droplevel()
    df_2['Time'] = str(time_2)
    df_2['Time_Adversario'] = str(time_1)
    df_2['Confronto'] = str(nome_final)
    df_2['Data'] = str(data)
    df_2['Estadio'] = str(estadio)
    df_2['Torcida'] = torcida


    #APPENDING
    df_3 = df_1.append(df_2)

    #escrevendo em excelfile
    writer = pd.ExcelWriter(nome_final+'.xlsx')
    df_3.to_excel(writer,"Estatisticas")
    writer.save()


In [7]:
def tratamento(local,nome):    
    entries = os.listdir()

    base = {}

    base = pd.DataFrame(base)


    for i in entries:
        if "xlsx" in i:
            base = base.append(pd.read_excel(i))
            base = base.drop_duplicates()
            writer = pd.ExcelWriter("C:/Users/mathe/Codes/Italiano_campeonato/"+nome+ ".xlsx")
            base.to_excel(writer,nome)
            writer.save()
        #escrevendo em excelfile
        else:
            pass
        
    
def limpeza(local,nome):
    entries = os.listdir()
    for i in entries:
        if "xlsx" in i:
            if nome in i:
                    pass
            else:
                os.remove(i)
        else:
            pass
    

    

In [8]:
start = time.time()

italiano_atual = ScoresFixtures("https://fbref.com/en/comps/11/schedule/Serie-A-Fixtures")

for url in italiano_atual.table:
    planilhas(url)
    

print('Duration: {} seconds'.format(time.time() - start))

KeyboardInterrupt: 

In [67]:
tratamento('C:/Users/mathe/Codes/Italiano_campeonato',"ITALIANO_2020")
limpeza('C:/Users/mathe/Codes/Italiano_campeonato',"ITALIANO_2020")

In [4]:
italiano_atual = ScoresFixtures("https://fbref.com/en/comps/11/schedule/Serie-A-Fixtures")

italiano_atual.table

['/en/matches/2b01c68a/Parma-Juventus-August-24-2019-Serie-A">Match Report',
 '/en/matches/de16a772/Fiorentina-Napoli-August-24-2019-Serie-A">Match Report',
 '/en/matches/5d32941f/Udinese-Milan-August-25-2019-Serie-A">Match Report',
 '/en/matches/3ac8069d/Roma-Genoa-August-25-2019-Serie-A">Match Report',
 '/en/matches/6237b77f/Torino-Sassuolo-August-25-2019-Serie-A">Match Report',
 '/en/matches/0326af7e/SPAL-Atalanta-August-25-2019-Serie-A">Match Report',
 '/en/matches/a8b82bb0/Hellas-Verona-Bologna-August-25-2019-Serie-A">Match Report',
 '/en/matches/de72f334/Sampdoria-Lazio-August-25-2019-Serie-A">Match Report',
 '/en/matches/fb06cc65/Cagliari-Brescia-August-25-2019-Serie-A">Match Report',
 '/en/matches/813ea7be/Internazionale-Lecce-August-26-2019-Serie-A">Match Report',
 '/en/matches/ba549b1e/Bologna-SPAL-August-30-2019-Serie-A">Match Report',
 '/en/matches/27c01148/Milan-Brescia-August-31-2019-Serie-A">Match Report',
 '/en/matches/14afe9c6/Juventus-Napoli-August-31-2019-Serie-A">Ma